In [12]:
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

apitoken = '0ac08bc0e23b22dcff4888822d56241be15b0a64e77e94c42639a524404aafbc7df008ab6043e4706b3b82aa42f03c06464b3c253b92ddf1337e80048cf53be0'
IBMQ.save_account(apitoken, overwrite = True)

# Setting the random seed
np.random.seed(seed=7)# You may choose any seed instead of just `0'
n = 4 # This is the number of bits in the string
alice_bits = randint(2, size=n)
print(alice_bits)

[1 0 1 0]


In [13]:
def callBellPair(circuit, h_qubit, x_qubit):
    circuit.h(q[h_qubit])
    circuit.cx(q[h_qubit], q[x_qubit])
    
    return circuit

def encodeMsg(circuit, qubit, msg):
    if len(msg) > 6:
        raise ValueError(f"message '{msg}' is invalid")
    
    if msg[0] == 1:
        circuit.z(q[qubit])
    
    if msg[1] == 1:
        circuit.x(q[qubit])
        
    if msg[2] == 1:
        circuit.z(q[qubit+2])
    
    if msg[3] == 1:
        circuit.x(q[qubit+2])
        
    circuit.id(q[qubit])
    circuit.id(q[qubit+2])
        
    return circuit

def decodeMsg(circuit, h_qubit, x_qubit):
    circuit.cx(q[h_qubit], q[x_qubit])
    circuit.h(q[h_qubit])
    
    return circuit

In [14]:
alice_bits = np.array([
    [0, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 1, 1, 0],
    [1, 1, 0, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 1],
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 1],
    [1, 1, 1, 1],
])

# For IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.providers.ibmq import least_busy
# Load local account information
IBMQ.load_account()
# Get the least busy backend
provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters=lambda x:
x.configuration().n_qubits >= 2
and not x.configuration().simulator
and x.status().operational==True))
print("least busy backend: ", backend)

shots = 1024

for alice in alice_bits:
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q, c)
    
    circuit = callBellPair(circuit, 1, 0)
    circuit = callBellPair(circuit, 3, 2)
    circuit.barrier()
    circuit = encodeMsg(circuit, 0, alice)
    circuit.barrier()
    circuit = decodeMsg(circuit, 0, 1)
    circuit = decodeMsg(circuit, 2, 3)
    
    circuit.barrier()
    circuit.measure(q, c)
    
    # For qsam_simulator
#     simulator = Aer.get_backend('qasm_simulator')
#     job = execute(circuit, simulator, shots = shots)
    
    # For IBMQ
    job = execute(circuit, backend = qcomp, shots = shots)
    job_monitor(job)

    result = job.result()
    counts = result.get_counts(circuit)
    
    # Flip alice to check accuracy
    message = np.flip(alice)
    correct_result = counts[''.join(map(str, message))]
    accuracy = (correct_result/shots)*100
    
    message = np.flip(alice)
    message = ''.join(map(str, message))

    first_qubit = message[:2]
    second_qubit = message[2:4]

    BER = 0
    SER = 0

    for state, count in counts.items():
        correct_bit = 0
        correct_symbol = 0
        for i in range(len(state)):
            if state[i] == message[i]:
                correct_bit += 1
        BER += (correct_bit/len(state)*count)

        if state[:2] == first_qubit:
            correct_symbol += 1

        if state[2:4] == second_qubit:
            correct_symbol += 1

        SER += (correct_symbol/(len(state)/2)*count)
        
    print(f"alice send: {alice} ||", f"bob get: {counts}", "==>", f"Accuracy: {accuracy:.2f}%")
    print(f"SER: {(1-SER/shots)*100:.2f}%", f"BER: {(1-BER/shots)*100:.2f}%")
    print("================================================================================")
    
# circuit.draw()

ibmqfactory.load_account:WARNING:2021-10-20 20:36:59,559: Credentials are already in use. The existing account in the session will be replaced.


least busy backend:  ibmq_lima
Job Status: job has successfully run
alice send: [0 0 0 0] || bob get: {'0000': 353, '0001': 11, '0010': 3, '0011': 3, '0100': 189, '0101': 5, '0110': 2, '0111': 1, '1000': 213, '1001': 4, '1010': 3, '1011': 1, '1100': 233, '1101': 1, '1110': 1, '1111': 1} ==> Accuracy: 34.47%
SER: 33.69% BER: 22.75%
Job Status: job has successfully run
alice send: [0 0 0 1] || bob get: {'0000': 156, '0010': 2, '0100': 203, '0101': 1, '0110': 1, '0111': 1, '1000': 376, '1001': 3, '1010': 6, '1011': 2, '1100': 266, '1101': 2, '1110': 2, '1111': 3} ==> Accuracy: 36.72%
SER: 32.23% BER: 21.29%
Job Status: job has successfully run
alice send: [0 0 1 0] || bob get: {'0000': 193, '0001': 2, '0010': 2, '0011': 1, '0100': 379, '0101': 3, '0110': 4, '0111': 1, '1000': 168, '1001': 2, '1010': 4, '1011': 2, '1100': 253, '1101': 4, '1110': 2, '1111': 4} ==> Accuracy: 37.01%
SER: 32.62% BER: 20.80%
Job Status: job has successfully run
alice send: [0 1 0 0] || bob get: {'0000': 9, '000